In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
def clean_data(path, tvt, csv):
    df = pd.read_csv(path + csv)
    filters = (df['cancelled']==0) & (df['dup']=='N')
    # Filter out cancelled flight, duplicate record. Drop 'cancelled', 'dup', 'icao24'
    df = df[filters].drop(['cancelled', 'dup', 'icao24'], axis=1)
    # Fill null values for 'weather_delay', 'late_aircraft_delay', 'dep_delay' with 0             note: after check, null for dep_delay means airplane departured on time.
    df[['dep_delay', 'weather_delay', 'late_aircraft_delay']] = df[['dep_delay', 'weather_delay', 'late_aircraft_delay']].fillna(0.0)
    # Drop rows that contain null values in 'arr_time'
    df.dropna(subset=['arr_time'], inplace=True)
    # Fill null values in 'arr_delay' by substracting 'crs_arr_time' from 'arr_time' 
    # (condition applied to adjust for hhmm format and arrive in next day issue)
    float_to_min = lambda x: x.astype(str).str[:-2].str.zfill(4).str[:2].astype(int) * 60 + x.astype(str).str[:-2].str.zfill(4).str[2:].astype(int)
    int_to_min = lambda x: x.astype(str).str.zfill(4).str[:2].astype(int) * 60 + x.astype(str).str.zfill(4).str[2:].astype(int)

    idx = df.loc[(df['arr_delay'].isna()) & (df['arr_time']<1000) & (df['crs_arr_time']>1800), 'arr_delay'].index
    df.loc[(df['arr_delay'].isna()) & (df['arr_time']<1000) & (df['crs_arr_time']>1800), 'arr_delay'] = (float_to_min(df['arr_time'])+24*60\
                                                                                                        - int_to_min(df['crs_arr_time']))[idx].values

    df['arr_delay'] = df['arr_delay'].fillna(float_to_min(df['arr_time']) - int_to_min(df['crs_arr_time']))
    # Combine same brand of manufactures
    manufactures = {
        'Embraer S A': 'Embraer',
        'Embraer-empresa Brasileira De': 'Embraer',
        'Mcdonnell Douglas Corporation': 'Mcdonnell Douglas',
        'Mcdonnell Douglas Aircraft Co': 'Mcdonnell Douglas',
        'Airbus Industrie': 'Airbus',
        'Boeing Of Canada/dehav Div': 'Boeing',
        'Saab-scania': 'Saab',
        'Gulfstream Aerospace Corp': 'Gulfstream Aerospace',
        'Bombardier Inc': 'Bombardier'
    }
    df['manufacturername'].replace(manufactures, inplace=True)
    # One hot encode the 'manufacturername'
    enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
    encoded = enc.fit_transform(df['manufacturername'].to_numpy().reshape(-1,1))
    ohe_df = pd.DataFrame(encoded, columns=enc.get_feature_names(['man']))
    df = pd.concat([df.reset_index(), ohe_df.reset_index()], axis=1).drop(['index', 'manufacturername', 'man_nan'], axis=1)
    # Create 'aircraft_age' and fillna with mean value
    df['aircraft_age'] = df['fl_date'].str[:4].astype(float) - df['built'].str[:4].astype(float)
    df['aircraft_age'] = df['aircraft_age'].fillna(int(df['aircraft_age'].mean()))
    df['aircraft_age'] = df['aircraft_age'].astype(int)
    df = df[(df['aircraft_age'] >= 0) & (df['aircraft_age'] <= 100)] # Drop outliers of 'aircraft_age' caused by source aircraft data.
    df.drop(['built'], axis=1, inplace=True)
    df = pd.get_dummies(df, columns=["type"], drop_first=True)
    print(f'\nIs there any null values now: {df.isnull().any().any()}')
    df.to_csv(f'{path}cleanedtest_{csv}', index=None)

In [3]:
def clean_test_data(path, tvt, csv):
    df = pd.read_csv(path + csv)
    if tvt == 'test_':
        df.drop(['dep_time', 'dep_delay', 'arr_time', 'weather_delay', 'late_aircraft_delay'], axis=1) 
    filters = (df['dup']=='N')
    # Filter out cancelled flight, duplicate record. Drop 'cancelled', 'dup', 'icao24'
    df = df[filters].drop(['dup', 'icao24'], axis=1)
    # Combine same brand of manufactures
    manufactures = {
        'Embraer S A': 'Embraer',
        'Embraer-empresa Brasileira De': 'Embraer',
        'Mcdonnell Douglas Corporation': 'Mcdonnell Douglas',
        'Mcdonnell Douglas Aircraft Co': 'Mcdonnell Douglas',
        'Airbus Industrie': 'Airbus',
        'Boeing Of Canada/dehav Div': 'Boeing',
        'Saab-scania': 'Saab',
        'Gulfstream Aerospace Corp': 'Gulfstream Aerospace',
        'Bombardier Inc': 'Bombardier'
    }
    df['manufacturername'].replace(manufactures, inplace=True)
    # One hot encode the 'manufacturername'
    enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
    encoded = enc.fit_transform(df['manufacturername'].to_numpy().reshape(-1,1))
    ohe_df = pd.DataFrame(encoded, columns=enc.get_feature_names(['man']))
    df = pd.concat([df.reset_index(), ohe_df.reset_index()], axis=1).drop(['index', 'manufacturername', 'man_nan'], axis=1)
    # Create 'aircraft_age' and fillna with mean value
    df['aircraft_age'] = df['fl_date'].str[:4].astype(float) - df['built'].str[:4].astype(float)
    df['aircraft_age'] = df['aircraft_age'].fillna(int(df['aircraft_age'].mean()))
    df['aircraft_age'] = df['aircraft_age'].astype(int)
    df = df[(df['aircraft_age'] >= 0) & (df['aircraft_age'] <= 100)] # Drop outliers of 'aircraft_age' caused by source aircraft data.
    df.drop(['built'], axis=1, inplace=True)
    df = pd.get_dummies(df, columns=["type"], drop_first=True)
    print(f'\nIs there any null values now: {df.isnull().any().any()}')
    df.to_csv(f'{path}cleanedtest_{csv}', index=None)

In [4]:
path = 'final2/Training/'
train = 'train_'
test = 'test_'
file1 = 'finalraw_flights_Dec_Jan.csv'
file2 = 'finalraw_flights_Dec.csv' 
file3 = 'finalraw_flights_Jan_1w.csv' 
file4 = 'finalraw_flights_Jan_2w.csv' 
file5 = 'finalraw_flights_Jan.csv' 
files = [file1,file2,file3,file4,file5]

In [5]:
for i in files:
    clean_data(path, train, i)

/home/zarish/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



Is there any null values now: False


/home/zarish/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



Is there any null values now: False


/home/zarish/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



Is there any null values now: False


/home/zarish/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



Is there any null values now: False


/home/zarish/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



Is there any null values now: False


In [9]:
path = 'final2/Validation/'
for i in files:
    clean_test_data(path, test, i)

/home/zarish/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



Is there any null values now: True


/home/zarish/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



Is there any null values now: True


/home/zarish/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



Is there any null values now: True


/home/zarish/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



Is there any null values now: True


/home/zarish/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



Is there any null values now: True


In [13]:
clean_test_data(path, 'pred_', 'finalraw_flights_test.csv')

/home/zarish/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



Is there any null values now: True
